In [68]:
import gzip
import os
import pickle
import sys
import wget
import numpy as np

def vectorized_result(y):
    e = np.zeros((10, 1))
    e[y] = 1.0
    return e

def load_mnist():
    if not os.path.exists(os.path.join(os.curdir, "data_mnist")):
        os.mkdir(os.path.join(os.curdir, "data_mnist"))
        wget.download("http://deeplearning.net/data/mnist/mnist.pkl.gz", out="data_mnist")

    data_file = gzip.open(os.path.join(os.curdir, "data_mnist", "mnist.pkl.gz"), "rb")
    train_data, val_data, test_data = pickle.load(data_file, encoding="latin1")
    data_file.close()

    train_inputs = np.array([np.reshape(x, (784, 1)) for x in train_data[0]])
    train_results = np.array([vectorized_result(y) for y in train_data[1]])
    train_data  = train_inputs.squeeze().T, \
    train_results.squeeze().T
    
    val_inputs = np.array([np.reshape(x, (784, 1)) for x in val_data[0]])
    val_results = np.array([vectorized_result(y) for y in val_data[1]])
    val_data  = val_inputs.squeeze().T, \
    val_results.squeeze().T
    
    test_inputs = np.array([np.reshape(x, (784, 1)) for x in test_data[0]])
    test_results = np.array([vectorized_result(y) for y in test_data[1]])
    test_data  = test_inputs.squeeze().T, \
    test_results.squeeze().T
    
    return train_data, val_data, test_data

In [69]:
from tensorflow import keras
import matplotlib.pyplot as plt



def load_dataset(flatten=True):
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    # normalize x
    X_train = X_train.astype(float) / 255.
    X_test = X_test.astype(float) / 255.

    # we reserve the last 10000 training examples for validation
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]
    
    y_train = np.array([vectorized_result(y) for y in y_train]).squeeze().T
    y_val = np.array([vectorized_result(y) for y in y_val]).squeeze().T
    y_test = np.array([vectorized_result(y) for y in y_test]).squeeze().T
    if flatten:
        X_train = X_train.reshape([ -1, X_train.shape[0]])
        X_val = X_val.reshape([ -1, X_val.shape[0]])
        X_test = X_test.reshape([ -1, X_test.shape[0]])

    return X_train, y_train, X_val, y_val, X_test, y_test
  
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()


In [70]:
from tensorflow import keras
import matplotlib.pyplot as plt
train_data, val_data, test_data = load_mnist()

In [71]:
train_x , train_y = train_data
val_x , val_y = val_data
test_x , test_y = test_data
#indices = np.random.permutation(len(train_x[1]))
#train_x , train_y = train_x[:,indices] , train_y[:,indices]
#val_x , val_y = val_data
#test_x , test_y = test_data

In [72]:
from main_class import *
from layers import *
from loss_functions import *

In [73]:
md=NeuralNetwork(MultiClassCrossEntropy)
np.random.seed(1)

n_x = 784    # num_px * num_px * 3

md.add(Dense(100, input_shape=(n_x,), initializer = 'normal'))
md.add(Activation('relu'))

md.add(Dense(200, initializer = 'normal'))
md.add(Activation('relu'))

md.add(Dense(10, initializer = 'normal'))
md.add(Activation_SoftMax())


In [74]:
hist = md.fit(train_x, train_y, n_epochs=25, batch_size=32)

100%|██████████| 25/25 [00:40<00:00,  1.60s/it]


In [79]:
def softmax(x):
        e_x = np.exp(x )
        return e_x / np.sum(e_x, axis=0, keepdims=True)
    
def accuracy(test_x, test_y):
    preds = md.predict(test_x)
    preds = np.array([y for y in np.argmax(preds, axis=0)]).squeeze()
    test_y_ = np.array([y for y in np.argmax(test_y, axis=0)]).squeeze()
    return np.mean(preds == test_y_)
accuracy(test_x , test_y)

0.9764

In [76]:
md=NeuralNetwork(SoftmaxCrossEntropy)
np.random.seed(1)

n_x = 784    # num_px * num_px * 3
md.add(Dense(100, input_shape=(n_x,), initializer = 'normal'))
md.add(Activation('relu'))
md.add(Dense(200, initializer = 'normal'))
md.add(Activation('relu'))
md.add(Dense(10, initializer = 'normal'))


In [77]:
hist = md.fit(train_x, train_y, n_epochs=25, batch_size=32)

100%|██████████| 25/25 [00:39<00:00,  1.57s/it]


In [78]:
accuracy(test_x , test_y)

0.9764